In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout
from keras.applications import VGG16
from keras.optimizers import Adam

# Частина 2

In [2]:
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

In [3]:
image_size = (32, 32)
train_images = tf.image.resize(train_images[..., np.newaxis], image_size)
train_images = np.repeat(train_images, 3, axis=-1)
test_images = tf.image.resize(test_images[..., np.newaxis], image_size)
test_images = np.repeat(test_images, 3, axis=-1)
train_images = np.array(train_images) / 255.0
test_images = np.array(test_images) / 255.0

In [4]:
num_classes = 10

train_labels = tf.keras.utils.to_categorical(train_labels, num_classes)
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes)

In [5]:
vgg_base = VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(image_size[0], image_size[1], 3)
    )


vgg_base.trainable = False

58889256/58889256 [==============================] - 9s 0us/step


In [6]:
model = Sequential([
    vgg_base,
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.25),
    Dense(num_classes, activation='softmax'),
])

In [7]:
optimizer = Adam(learning_rate=0.001)

model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [8]:
batch_size = 128
epochs = 20

history = model.fit(train_images, train_labels,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(test_images, test_labels))

Epoch 1/20
469/469 [==============================] - 421s 895ms/step - loss: 0.5875 - accuracy: 0.7932 - val_loss: 0.4603 - val_accuracy: 0.8347
Epoch 2/20
469/469 [==============================] - 415s 886ms/step - loss: 0.4259 - accuracy: 0.8431 - val_loss: 0.4068 - val_accuracy: 0.8508
Epoch 3/20
469/469 [==============================] - 484s 1s/step - loss: 0.3914 - accuracy: 0.8554 - val_loss: 0.4102 - val_accuracy: 0.8512
Epoch 4/20
469/469 [==============================] - 488s 1s/step - loss: 0.3688 - accuracy: 0.8637 - val_loss: 0.3791 - val_accuracy: 0.8603
Epoch 5/20
469/469 [==============================] - 530s 1s/step - loss: 0.3518 - accuracy: 0.8691 - val_loss: 0.3725 - val_accuracy: 0.8666
Epoch 6/20
469/469 [==============================] - 532s 1s/step - loss: 0.3397 - accuracy: 0.8745 - val_loss: 0.3720 - val_accuracy: 0.8639
Epoch 7/20
469/469 [==============================] - 496s 1s/step - loss: 0.3254 - accuracy: 0.8790 - val_loss: 0.3615 - val_accuracy: 

In [9]:
_, accuracy = model.evaluate(test_images, test_labels)
print(f'Точність на тестових даних: {accuracy * 100:.2f}%')

313/313 [==============================] - 62s 197ms/step - loss: 0.3498 - accuracy: 0.8800
Точність на тестових даних: 88.00%


In [10]:
vgg_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 8, 8, 128)         0     

In [11]:
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
vgg_base.trainable = True
set_trainable = False
for layer in vgg_base.layers:
    if layer.name == "block5_conv1":
        set_trainable = True
    if layer.name == "block5_conv2":
        set_trainable = True
    if layer.name == "block5_conv3":
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False


modified_model = Sequential([
   vgg_base,
   Flatten(),
   Dense(256, activation="relu"),
   Dense(10, activation="softmax"),
])

In [12]:
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy'])

history = model.fit(train_images, train_labels,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(test_images, test_labels))

Epoch 1/20
469/469 [==============================] - 436s 927ms/step - loss: 0.2034 - accuracy: 0.9245 - val_loss: 0.3368 - val_accuracy: 0.8863
Epoch 2/20
469/469 [==============================] - 434s 926ms/step - loss: 0.1971 - accuracy: 0.9278 - val_loss: 0.3377 - val_accuracy: 0.8849
Epoch 3/20
469/469 [==============================] - 433s 924ms/step - loss: 0.1927 - accuracy: 0.9294 - val_loss: 0.3376 - val_accuracy: 0.8864
Epoch 4/20
469/469 [==============================] - 435s 928ms/step - loss: 0.1908 - accuracy: 0.9305 - val_loss: 0.3361 - val_accuracy: 0.8859
Epoch 5/20
469/469 [==============================] - 433s 924ms/step - loss: 0.1894 - accuracy: 0.9316 - val_loss: 0.3377 - val_accuracy: 0.8862
Epoch 6/20
469/469 [==============================] - 435s 927ms/step - loss: 0.1880 - accuracy: 0.9315 - val_loss: 0.3352 - val_accuracy: 0.8859
Epoch 7/20
469/469 [==============================] - 434s 926ms/step - loss: 0.1877 - accuracy: 0.9316 - val_loss: 0.3369 -

In [13]:
_, accuracy = model.evaluate(test_images, test_labels)
print(f'Точність на тестових даних: {accuracy * 100:.2f}%')

313/313 [==============================] - 65s 209ms/step - loss: 0.3403 - accuracy: 0.8882
Точність на тестових даних: 88.82%
